# ROBUST PROPERTY-STRUCTURE LINKAGE FOR POLYMER COMPOSITES



In [1]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt


Model parameters 

In [2]:
sample_size = 200
n_samples = 6 * [sample_size]
size = (101, 101)
elastic_modulus = (1.3, 75)
poissons_ratio = (0.42, .22)
macro_strain = 0.001
n_phases = 2
grain_size = [(40, 2), (10, 2), (2, 40), (2, 10), (2, 30), (30, 2)]
v_frac = [(0.8, 0.2), (0.7, 0.3), (0.6, 0.4), (0.5, 0.5), (0.3, 0.7), (0.4, 0.6)]
per_ch = 0.1
seed=10

## FE Simulation

Create Data from FE Simulation

In [3]:
from pymks.datasets.elastic_FE_simulation import ElasticFESimulation

sim = ElasticFESimulation(poissons_ratio=poissons_ratio,
                          elastic_modulus=elastic_modulus, macro_strain=macro_strain)

/home/grdstu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/grdstu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/grdstu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/grdstu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Generating Structures

In [4]:
from pymks.datasets import make_microstructure
dataset = np.concatenate([make_microstructure(n_samples=samps,
                              size=size, grain_size=grains, seed=seed, 
                               volume_fraction=v_f, percent_variance=per_ch)
                          for samps, grains, v_f in zip(n_samples, grain_size, v_frac)])
dataset.shape

/home/grdstu/anaconda3/lib/python3.6/site-packages/pymks/datasets/microstructure_generator.py:75: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  X_bool = X_blur[..., None] > seg_values[seg_slices]


(1200, 101, 101)

## Run FE Simulation

In [9]:
sim.run(dataset)
local_strain = sim.response
print(local_strain.shape)

(1200L, 101L, 101L)




Now we are going to print out a few microstructres to look at how the fiber length, orientation and volume fraction are varied.  

In [11]:
from pymks import PrimitiveBasis

p_basis = PrimitiveBasis(2)

X_ = p_basis.discretize(dataset)
index = tuple([None for i in range(len(size) + 1)]) + (slice(None),)
modulus = np.sum(X_ * np.array(elastic_modulus)[index], axis=-1)
y_stress_field = local_strain * modulus
stresses = np.average(y_stress_field.reshape(len(y_stress_field), -1), axis=1)
print stresses

[  5.71442757   5.46860336   6.49412375 ...,  34.27138828  32.15890794
  32.33349239]


In [12]:
print np.average(stresses)

10.5129894018


In [13]:
moduli = stresses / macro_strain

## Creating the Model


In [14]:
from pymks import MKSHomogenizationModel
from pymks import PrimitiveBasis

p_basis = PrimitiveBasis()
model = MKSHomogenizationModel(basis=p_basis, correlations=[(0, 0), (1, 1)], periodic_axes=[0, 1])


## Split Data

In [38]:
from sklearn.cross_validation import train_test_split


flat_shape = (dataset.shape[0],) + (dataset[0].size,)
data_train, data_test, moduli_train, moduli_test = train_test_split(
    dataset.reshape(flat_shape), moduli, test_size=0.2, random_state=3)


## Search Parameter Space

In [16]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

params_to_tune = {'degree': np.arange(1, 7), 'n_components': np.arange(1, 11)}
fit_params = {'size': dataset[0].shape}
gs = GridSearchCV(model, params_to_tune, fit_params=fit_params, scoring=mse_scorer).fit(data_train, moduli_train)


Let's take a look at the results.

In [17]:
print('Order of Polynomial'), (gs.best_estimator_.degree)
print('Number of Components'), (gs.best_estimator_.n_components)
print('Score'), (gs.score(data_test, moduli_test))


Order of Polynomial 3
Number of Components 3
Score 0.999144105047


## Use best Model

In [39]:
model = gs.best_estimator_
model.fit(data_train.reshape((data_train.shape[0],) + dataset.shape[1:]), moduli_train)

### Percent Error 

In [40]:
moduli_predict = model.predict(data_test)
_moduli_train = model.predict(data_train.reshape((data_train.shape[0],) + dataset.shape[1:]))
pred_percent_error = 100 * np.abs(moduli_predict - moduli_test) / moduli_test
train_percent_error = 100 * np.abs(_moduli_train - moduli_train) / moduli_train
print 'Mean Predicted Percent Error', np.mean(pred_percent_error)
print 'Mean Training Percent Error', np.mean(train_percent_error)


Mean Predicted Percent Error 1.7163371734
Mean Training Percent Error 1.71528052294


## Other Models 

In [53]:
_data_test = data_test.reshape((data_test.shape[0],) + dataset.shape[1:])
Vf = np.zeros(data_test.shape[0])
for i in range(0, data_test.shape[0]):
    Vf[i] = sum(sum(_data_test[i]))/(101.0*101.0)

[ 0.69914714  0.39947064  0.69953926 ...,  0.60072542  0.70012744
  0.19949025]


In [69]:
testing_indices = []
for x in data_test:
    index = np.where((x == dataset.reshape((dataset.shape[0],
                                            -1))).all(axis=1))[0][0]
    testing_indices.append(index)

Halpin-Tsai

In [43]:
# l = average fiber length
# d = average fiber diameter
# grain_size = [(40, 2), (10, 2), (2, 40), (2, 10), (2, 30), (30, 2)]
d = np.zeros(dataset.shape[0])
l = np.zeros(dataset.shape[0])
l[0:200] = 40
d[0:200] = 2
l[200:400] = 10
d[200:400] = 2
l[400:600] = 2
d[400:600] = 40
l[600:800] = 2
d[600:800] = 10
l[800:1000] = 2
d[800:1000] = 30
l[1000:1200] = 30
d[1000:1200] = 2

In [60]:
l_test = np.array([l[i] for i in testing_indices])
d_test = np.array([d[i] for i in testing_indices])

[  2.   2.   2. ...,  30.   2.  40.]


In [63]:
Em, Ef = elastic_modulus

nu_l = ((Ef/Em)-1)/((Ef/Em)+(2*l_test/d_test))
# Logitudinal elastic modulus
El = Em*((1+(2*l_test/d_test)*nu_l*Vf)/(1-nu_l*Vf))

Halpin_Tsai_moduli = np.concatenate((moduli_test[None], El[None]))

Rule of mixtures

In [64]:
E_mix_u = (1-Vf)*Em+(Vf)*Ef
Mixtures_upper_moduli = np.concatenate((moduli_test[None], E_mix_u[None]))
E_mix_l = (Ef*Em)/(((1-Vf)*Ef)+(Vf*Em))
Mixtures_lower_moduli = np.concatenate((moduli_test[None], E_mix_l[None]))

### Compare Models


In [70]:
def _draw_goodness_of_fit(data, labels, legend_location=1):
    plt.close('all')
    y_total = data[0]
    y_min, y_max = np.min(y_total), np.max(y_total)
    middle = (y_max + y_min) / 2.
    data_range = y_max - y_min
    line = np.linspace(middle - data_range * 1.03 / 2,
                       middle + data_range * 1.03 / 2, endpoint=False)
    plt.plot(line, line, '-', linewidth=3, color='#000000')
    colors = ['#f46d43', '#1f78b4', '#e31a1c','#6a3d9a', '#b2df8a']
    for i in range(1):
        colors.remove(colors[2])
    for d, l, c in zip(data, labels, colors):
        plt.plot(d[0], d[1], 'o', color=c, label=l)
    plt.title('Goodness of Fit', fontsize=20)
    plt.xlabel('Actual Modulus (GPa)', fontsize=18)
    plt.ylabel('Predicted Modulus (GPa)', fontsize=18)
    plt.legend(loc=legend_location, fontsize=15)
    plt.show()

In [71]:

pred_data = np.array([moduli_test, moduli_predict])
_draw_goodness_of_fit([pred_data, Mixtures_upper_moduli, Mixtures_lower_moduli, Halpin_Tsai_moduli],
                      ['Model', 'Upper ROM', 'Lower ROM', 'Halpin-Tsai'],
                     legend_location=1)
